## Brainstorming
#### Analysis Questions 
- How many temperature readings in each month seemed inaccurate
    - I defined the outliers as being 2 standard deviations above or below the mean
- Relationship between solar radiation and rainfall for each month 
- Relationship between solar radiation and max temperature for each month 
- One table showing the mean of specific attributes for each month 

#### Data Quality Assessment
- **First**:
    - Renaming all the columns to a shorter, simplified name
    - Needing to use `FAWN_raw.columns = FAWN_raw.columns.str.strip()` to make all the white spaces between letters, numbers, or symbols single spaces so all the spaces are uniform  
- Check data types using `.info()`
    - displays: Column names, types, non-null counts, memory
- Check statistical description using `.describe()`  
- Use `.value_counts()` for frequency count of each category (**need to do**)
- Finding outliers for max and min temperatures   
    - find `std()` for max and min temperatures, by month, for all three years using `groupby()`  



## EDA Workflow


### Load and Initial Reconnaissance


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

# loading Dataset from a CSV file #
FAWN_raw_initial = pd.read_csv("data/FAWN_report.csv")
FAWN_raw = pd.read_csv("data/FAWN_report.csv", parse_dates=["Period"], date_format="%d-%b-%y")
FAWN_clean = FAWN_raw
# FAWN_raw_spaces = FAWN_raw.columns.tolist()

# Cleaning spaces in raw data set
FAWN_clean.columns = FAWN_raw.columns.str.replace("  ", " ")

# Quality Assessment
FAWN_clean.describe()
FAWN_clean_loc_counts = FAWN_clean["FAWN Station"].value_counts()
FAWN_clean_loc = FAWN_clean["FAWN Station"].unique()
print(f"\n{FAWN_clean.dtypes}\n{FAWN_clean.info()}\n\n{FAWN_clean_loc_counts}\n")
print(
    f"{FAWN_clean_loc}\n\nThere are {len(FAWN_clean_loc)} weather stations represented in this DataFrame.\n"
)
print(f"Shape/dimensions of DataFrame: {FAWN_clean.shape}\n")
print(f"Number of Elements: {FAWN_clean.size}\n")

FAWN_clean.head()

#### Data worth exploring using Analysis Questions 
- How many temperature readings in each month seemed inaccurate or could be classified as outliers?
- How Barametric data compares to Relative humidity by FAWN Station
- How Barametric data compares to Relative humidity by FAWN Station  
#### Other trends in the data to explore 
- Relationship between solar radiation and rainfall for each month
- Relationship between solar radiation and max temperature for each month





### Data Quality Assessment 
- The column names were in a scattered format and hard to read:
    - Renamed the columns to a more readable format
- The `N (# obs)` column could be worth exploring for accuracy purposes
  - Using `value_counts()`, displays the number of observations for each data value
  - This data is largely valid because the vast majority of individual days have 96 observations
    - *For reference*: The assumption is that data is collected every 15 minutes for 24 hours
    - This means that the columns with *average values* should be averaged over 24 hours in 15-minute intervals 
- All of the `dtypes` displayed above are accurate, and the DataFrame appears not to have any NaN values when comparing the result of `dtypes` to the `shape` of the DataFrame
- Used `groupby()` to separate the min temp by month and then used `quantile(0.5)`, which finds the temperature that 50% of the temperatures are above and below
- The columns: `2m Rain tot (in)` and `2m Rain max over 15min (in)` contain some values of 0
    - The count of these values can be found using `isin()` 
- Outliers can be determined with `between()`
  - could use by month 

### Data to graph 
- pie chart of `N_unique = FAWN_clean['N (# obs)'].value_counts()` for `N (# obs)`
- rainy and non-rainy days for each month, and FAWN Station using the variables `zero_rain_Monthcount` and `rain_Monthcount`
  - Shows how many rainy and non-rainy days there are for each month
  - one graph for each type of day 

In [ ]:
N_unique = FAWN_clean["N (# obs)"].value_counts()
print(f"{N_unique}")

# FAWN_clean['FAWN Station'].between()
# Grouping min temp by month to find quantile %s
FAWN_clean["Month"] = FAWN_clean["Period"].dt.month_name()
# FAWN_clean['2m T min (F)'].quantile(0.5)
monthyly_T_min = FAWN_clean.groupby("Month")["2m T min (F)"]
monthyly_T_min.quantile(0.5)

zero_rain_Monthcount = (
    FAWN_clean[FAWN_clean["2m Rain tot (in)"] == 0]
    .groupby(["Month", "FAWN Station"])
    .size()
    .unstack()
)
print(f"\nNumber of non-rainy days categorized by month: \n\n{zero_rain_Monthcount}\n")

rain_Monthcount = (
    FAWN_clean[FAWN_clean["2m Rain tot (in)"] != 0]
    .groupby(["Month", "FAWN Station"])
    .size()
    .unstack()
)
print(f"\nNumber of rainy days categorized by month: \n\n{rain_Monthcount}\n")

non_rainyDays = FAWN_clean[FAWN_clean["2m Rain tot (in)"].isin([0])]
rainyDays = FAWN_clean[~FAWN_clean["2m Rain tot (in)"].isin([0])]
print(f"\nNumber of rainy days: {len(rainyDays['2m Rain tot (in)'])}\n")
print(f"Number of non-rainy days: {len(non_rainyDays['2m Rain tot (in)'])}\n")

N_outliers = rain_Monthcount.index[5]
N_outliers

In [ ]:
# To create a pie chart for unique N observations
plt.figure(figsize=(9, 9))
plt.pie(N_unique.values, labels=N_unique.index, startangle=180)
plt.title("Distribution of N (# obs) Values", fontweight="bold")
plt.axis("equal")
plt.show()

# To get unique stations
stations = FAWN_clean["FAWN Station"].unique()

for station in stations:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # To compute Non-rainy days
    if station in zero_rain_Monthcount.columns:
        zero_rain_Monthcount[station].plot(kind="bar", ax=axes[0], color="red")
        axes[0].set_title(f"Non-Rainy days by Month - {station}", fontweight="bold")
        axes[0].set_xlabel("Month")
        axes[0].set_ylabel("Number of Days")
        axes[0].tick_params(axis="x", rotation=45)

    # To compute rainy days
    if station in rain_Monthcount.columns:
        rain_Monthcount[station].plot(kind="bar", ax=axes[1], color="steelblue")
        axes[1].set_title(f"Rainy days by Month - {station}", fontweight="bold")
        axes[1].set_xlabel("Month")
        axes[1].set_ylabel("Number of Days")
        axes[1].tick_params(axis="x", rotation=45)

    plt.tight_layout()
    plt.show()

### Cleaning Decisions 
- Reformatting and renaming columns
- In data quality assessment, we evaluated the `N (# obs)` cloumn by using `value_counts`
  - Based on the results, if the number of observations was below 87, this meant that these values were less than 90%` of the maximum number of observations, 96, and therefore they could skew the results and be labeled as **legitimate outliers** 
  - For this reason, we decided to remove these values
  - Used indexing to remove outliers
  - `FAWN_clean = FAWN_clean[FAWN_clean['N_obs'] >= 87]`
- Removing a total of 5 outliers based on observation values in the `N (# obs)` column introduces little bias because there are over 6500 total observations 

In [ ]:
# Renaming columns
FAWN_clean.rename(
    columns={
        "FAWN Station": "FAWN Station",
        "Period": "Period",
        "2m T avg (F)": "Temp_avg (F)",
        "2m T min (F)": "T_min (F)",
        "2m T max (F)": "T_max (F)",
        "2m DewPt avg (F)": "DewPt_avg (F)",
        "RelHum avg 2m (pct)": "RelHum_avg (pct)",
        "2m Rain tot (in)": "Rain_tot (in)",
        "2m Rain max over 15min (in)": "Rain_max over 15min (in)",
        "SolRad avg2m (w/m^2)": "SolRad_avg @ 2m (w/m^2)",
        "10m Wind avg (mph)": "Wind_avg @ 10m (mph)",
        "10m Wind min (mph)": "Wind_min @ 10m (mph)",
        "10m Wind max (mph)": "Wind_max @ 10m (mph)",
        "WDir avg10m (deg)": "Wind_Dir_avg @ 10m (deg)",
        "BP avg (mb)": "Barametric_Pre_avg (mb)",
        "N (# obs)": "N_obs",
        "2m WetBulb (F)": "WetBulb @ 2m (F)",
    },
    inplace=True,
)

# FAWN_raw = FAWN_raw.replace('0', np.nan)
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

# print(f"{FAWN_clean.isna().sum()}\n")
# print(f"{FAWN_raw_spaces}\n")
# print(f"{FAWN_raw.columns}\n")

# Removing legitimate outliers from the N (# obs) column
# N_outliers = FAWN_clean.query('`N_obs` < 87')
# N_outliers = FAWN_clean[FAWN_clean['N_obs'] < 87]
# print(N_outliers)
FAWN_clean = FAWN_clean[FAWN_clean["N_obs"] >= 87]
print(
    f"Observation outliers removed, shown by Shape of DataFrame.\nNew shape: {FAWN_clean.shape}\n"
)

# df.query('price > @min_price and quantity < 100')
FAWN_clean.head()

### Statistical EDA
- `.describe()` is used to display descriptive statistics of each column with numerical values
  - `.drop()` excludes columns containing strings
 
### Data to graph 
- A scatterplot of Barametric data on the x-axis and Relative humidity on the y-axis **by** FAWN Station
  - This will help us determine if we can predict windspeed using barometric pressure
- A scatterplot of Barametric data on the x-axis and Relative humidity on the y-axis **by** FAWN Station
  - This will help us determine if we can predict rainfall using barometric pressure
I, Daniel, work for the Air Force, and this data can help my team predict favorable conditions for flying drones
- These are good trends in the data as well to graph 
  - Relationship between solar radiation and rainfall for each month
  - Relationship between solar radiation and max temperature for each month


In [ ]:
# Displaying descriptive statistics of each column with numerical values
print(f"{FAWN_clean.drop(columns=['FAWN Station', 'Period']).describe().T}\n")

# print(f"{FAWN_clean.groupby(['FAWN Station', 'Month']).describe().T}")

# Finding min temp outliers by month
FAWN_clean["Month"] = FAWN_clean["Period"].dt.month_name()
monthyly_std_T_min = FAWN_clean.groupby(["Month"])["T_min (F)"].std() * 2
monthyly_mean_T_min = FAWN_clean.groupby(["Month"])["T_min (F)"].mean()
print(f"Two times Std: \n{monthyly_std_T_min}\n")
print(f"Mean of each month: \n{monthyly_mean_T_min}\n")

# Creating a separate column for Days
FAWN_clean["Day"] = FAWN_clean["Period"].dt.day_name()
# daily_T_min_byMonth = FAWN_clean.groupby('Month').agg({'Day':

# Calculating two std dev values above and below the mean for each month
two_std_overMean = monthyly_std_T_min + monthyly_mean_T_min
two_std_underMean = monthyly_mean_T_min - monthyly_std_T_min
print(f"Two std above Mean: \n{two_std_overMean}\n")
print(f"Two std below Mean: \n{two_std_underMean}\n")

"""FAWN_clean.merge(
    two_std_underMean.rename('Two std above Temp_min Mean'), 
    left_on=FAWN_clean['Month'], 
    right_index=True, 
    how='left'
)"""

# Adding two std dev values above and below the mean for each month
FAWN_clean["Two std above Temp_min Mean"] = FAWN_clean["Month"].map(two_std_overMean)
FAWN_clean["Two std below Temp_min Mean"] = FAWN_clean["Month"].map(two_std_underMean)

# Setting Period as index
# FAWN_clean = FAWN_clean.reset_index()
# FAWN_clean = FAWN_clean.set_index('Period')


# Finding Outliers
Temp_min_outlier = FAWN_clean[
    (FAWN_clean["T_min (F)"] > FAWN_clean["Two std above Temp_min Mean"])
    | (FAWN_clean["T_min (F)"] < FAWN_clean["Two std below Temp_min Mean"])
]
# print(Temp_min_outlier.count())

# FAWN_clean.groupby()
# print(f"\nNumber of outliers: \n{Temp_min_outlier['T_min (F)']}")
FAWN_clean

In [ ]:
# SCATTERPLOT 1: Barometric Pressure vs Relative Humidity by FAWN Station
plt.figure(figsize=(10, 6))
for station in stations:
    station_data = FAWN_clean[FAWN_clean["FAWN Station"] == station]
    plt.scatter(
        station_data["Barametric_Pre_avg (mb)"],
        station_data["RelHum_avg (pct)"],
        label=station,
        alpha=0.6,
        s=50,
    )

plt.xlabel("Barometric Pressure (mb)", fontsize=12, fontweight="bold")
plt.ylabel("Relative Humidity (%)", fontsize=12, fontweight="bold")
plt.title(
    "Scatterplot: Barometric Pressure vs Relative Humidity by FAWN Station",
    fontsize=13,
    fontweight="bold",
)
plt.legend(title="FAWN Station", loc="best")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# SCATTERPLOT 2: Barometric Pressure vs Relative Humidity (for rainfall prediction)
# Separate visualization with color coding for different stations
fig, ax = plt.subplots(figsize=(10, 6))
for station in stations:
    station_data = FAWN_clean[FAWN_clean["FAWN Station"] == station]
    ax.scatter(
        station_data["Barametric_Pre_avg (mb)"],
        station_data["RelHum_avg (pct)"],
        label=station,
        alpha=0.5,
        s=60,
        edgecolors="black",
        linewidth=0.5,
    )

ax.set_xlabel("Barometric Pressure (mb)", fontsize=12, fontweight="bold")
ax.set_ylabel("Relative Humidity (%)", fontsize=12, fontweight="bold")
ax.set_title(
    "Barometric Pressure vs Relative Humidity for Rainfall Prediction\n(by FAWN Station)",
    fontsize=13,
    fontweight="bold",
)
ax.legend(title="FAWN Station", loc="best", fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# RELATIONSHIP 1: Solar Radiation vs Rainfall (by month)
# Group by month
monthly_data = (
    FAWN_clean.groupby("Month")
    .agg({"SolRad_avg @ 2m (w/m^2)": "mean", "Rain_tot (in)": "mean"})
    .reset_index()
)

fig, ax1 = plt.subplots(figsize=(10, 6))

# To plot solar radiation on primary y-axis
color = "tab:orange"
ax1.set_xlabel("Month", fontsize=12, fontweight="bold")
ax1.set_ylabel("Average Solar Radiation (w/m²)", color=color, fontsize=12, fontweight="bold")
ax1.plot(
    monthly_data["Month"],
    monthly_data["SolRad_avg @ 2m (w/m^2)"],
    marker="o",
    linewidth=2,
    markersize=8,
    label="Solar Radiation",
    color=color,
)
ax1.tick_params(axis="y", labelcolor=color)
ax1.grid(True, alpha=0.3)

# let's create a secondary y-axis for rainfall
ax2 = ax1.twinx()
color = "tab:blue"
ax2.set_ylabel("Average Rainfall (in)", color=color, fontsize=12, fontweight="bold")
ax2.plot(
    monthly_data["Month"],
    monthly_data["Rain_tot (in)"],
    marker="s",
    linewidth=2,
    markersize=8,
    label="Rainfall",
    color=color,
)
ax2.tick_params(axis="y", labelcolor=color)

plt.title(
    "Relationship between Solar Radiation and Rainfall by Month", fontsize=13, fontweight="bold"
)
ax1.set_xticks(monthly_data["Month"])
ax1.set_xticklabels(
    ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
)

# Add legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="best", fontsize=10)

plt.tight_layout()
plt.show()

# RELATIONSHIP 2: Solar Radiation vs Max Temperature (by month)
monthly_temp_data = (
    FAWN_clean.groupby("Month")
    .agg({"SolRad_avg @ 2m (w/m^2)": "mean", "T_max (F)": "mean"})
    .reset_index()
)

fig, ax1 = plt.subplots(figsize=(10, 6))

# To plot solar radiation on primary y-axis
color = "tab:blue"
ax1.set_xlabel("Month", fontsize=12, fontweight="bold")
ax1.set_ylabel("Average Solar Radiation (w/m²)", color=color, fontsize=12, fontweight="bold")
ax1.plot(
    monthly_temp_data["Month"],
    monthly_temp_data["SolRad_avg @ 2m (w/m^2)"],
    marker="o",
    linewidth=2,
    markersize=8,
    label="Solar Radiation",
    color=color,
)
ax1.tick_params(axis="y", labelcolor=color)
ax1.grid(True, alpha=0.3)

# let's create a secondary y-axis for max temperature
ax2 = ax1.twinx()
color = "tab:red"
ax2.set_ylabel("Average Max Temperature (°F)", color=color, fontsize=12, fontweight="bold")
ax2.plot(
    monthly_temp_data["Month"],
    monthly_temp_data["T_max (F)"],
    marker="s",
    linewidth=2,
    markersize=8,
    label="Max Temperature",
    color=color,
)
ax2.tick_params(axis="y", labelcolor=color)

plt.title(
    "Relationship between Solar Radiation and Max Temperature by Month",
    fontsize=13,
    fontweight="bold",
)
ax1.set_xticks(monthly_temp_data["Month"])
ax1.set_xticklabels(
    ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
)

# Add legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="best", fontsize=10)

plt.tight_layout()
plt.show()

#### Histogram of Minimum Temp values **by** month
- Graph the minimum temperature value for each day by month
- Should give you 12 graphs 
    - Label two std deviations above **and** below the mean on **each** graph 
- `Temp_min_outlier = FAWN_clean[ (FAWN_clean["T_min (F)"] > FAWN_clean["Two std above Temp_min Mean"])
    | (FAWN_clean["T_min (F)"] < FAWN_clean["Two std below Temp_min Mean"])]`
    - Try to use this variable, `Temp_min_outlier`, to highlight the outliers on the graph 


In [ ]:
# To create a column for the Temp_min_outlier for the entire dataset
# Calculate the 2 standard deviation boundaries for each month AND station
monthly_station_stat = (
    FAWN_clean.groupby(["Month", "FAWN Station"])["T_min (F)"].agg(["mean", "std"]).reset_index()
)
monthly_station_stat["upper_bound"] = monthly_station_stat["mean"] + 2 * monthly_station_stat["std"]
monthly_station_stat["lower_bound"] = monthly_station_stat["mean"] - 2 * monthly_station_stat["std"]

# Merge with the DataFrame
FAWN_clean = FAWN_clean.merge(
    monthly_station_stat[["Month", "FAWN Station", "upper_bound", "lower_bound"]],
    on=["Month", "FAWN Station"],
    how="left",
    suffixes=("", "_monthly_station"),
)

# Create the outlier column based on month AND station
FAWN_clean["Temp_min_outlier"] = (FAWN_clean["T_min (F)"] > FAWN_clean["upper_bound"]) | (
    FAWN_clean["T_min (F)"] < FAWN_clean["lower_bound"]
)

# Get unique stations and months
stations = sorted(FAWN_clean["FAWN Station"].unique())
months = sorted(FAWN_clean["Month"].unique())
month_names = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Create separate figure for each station
for station in stations:
    # Create subplots (3 rows x 4 columns for 12 months)
    fig, axes = plt.subplots(3, 4, figsize=(20, 15))
    axes = axes.flatten()

    # Loop through each month
    for idx, month in enumerate(months):
        ax = axes[idx]

        # Filter data for this month AND station
        month_station_data = FAWN_clean[
            (FAWN_clean["Month"] == month) & (FAWN_clean["FAWN Station"] == station)
        ].copy()

        # Get the temperature data
        temp_data = month_station_data["T_min (F)"].dropna()

        if len(temp_data) > 0:
            # Identify outliers and non-outliers
            outliers = month_station_data[month_station_data["Temp_min_outlier"] == True][
                "T_min (F)"
            ].dropna()
            non_outliers = month_station_data[month_station_data["Temp_min_outlier"] == False][
                "T_min (F)"
            ].dropna()

            # Calculate statistics
            mean_temp = temp_data.mean()
            std_temp = temp_data.std()

            # Create bins
            bin_edges = np.linspace(temp_data.min(), temp_data.max(), 20)

            # Histogram for non-outliers
            ax.hist(
                non_outliers,
                bins=bin_edges,
                alpha=0.6,
                color="blue",
                label="Non-outliers",
                density=True,
                edgecolor="black",
            )

            # Histogram for outliers (if any)
            if len(outliers) > 0:
                ax.hist(
                    outliers,
                    bins=bin_edges,
                    alpha=0.9,
                    color="red",
                    label="Outliers",
                    density=True,
                    edgecolor="darkred",
                    linewidth=1.5,
                )

            # Add normal distribution curve
            x = np.linspace(temp_data.min(), temp_data.max(), 100)
            normal_curve = stats.norm.pdf(x, mean_temp, std_temp)
            ax.plot(x, normal_curve, "green", linewidth=2, label="Normal distribution")

            # Add vertical line for the mean
            ax.axvline(
                mean_temp, color="black", linestyle="--", linewidth=1.5, alpha=0.7, label="Mean"
            )

            # Add boundaries (2 std deviations)
            upper_bound = mean_temp + 2 * std_temp
            lower_bound = mean_temp - 2 * std_temp
            ax.axvline(
                upper_bound,
                color="orange",
                linestyle="--",
                linewidth=1,
                alpha=0.5,
                label="Upper bound",
            )
            ax.axvline(
                lower_bound,
                color="red",
                linestyle="--",
                linewidth=1,
                alpha=0.5,
                label="Lower bound",
            )

            # Labels and title
            ax.set_xlabel("Temperature (°F)", fontsize=10, fontweight="bold")
            ax.set_ylabel("Density", fontsize=10, fontweight="bold")
            ax.set_title(
                f"{month_names[idx]}\n(n={len(temp_data)}, outliers={len(outliers)})",
                fontsize=11,
                fontweight="bold",
            )
            ax.legend(loc="upper right", fontsize=7)
            ax.grid(True, alpha=0.3)
        else:
            # No data for this month/station combination
            ax.text(0.5, 0.5, "No Data", ha="center", va="center", fontsize=14)
            ax.set_title(f"{month_names[idx]}", fontsize=11, fontweight="bold")

    # Overall title for the station
    plt.suptitle(
        f"Distribution of Minimum Temperature with Outliers by Month\nStation: {station} (2 Standard Deviations)",
        fontsize=16,
        fontweight="bold",
        y=0.998,
    )
    plt.tight_layout()
    plt.show()